In [1]:
# Packages for Web-Scraping
from requests import get
from bs4 import BeautifulSoup
from time import time
from time import sleep
from random import randint
from IPython.core.display import clear_output
from warnings import warn

# Packages for Saving File after Scraping
import numpy as np
import pandas as pd

In [8]:
# url 1st page info, response object, and sample
url = 'https://www.politifact.com/factchecks/list/?page=1&category=truth-o-meter'
response = get(url)
print(response.text[:500])


<!DOCTYPE html>
<html lang="en-US" dir="ltr">
<head>
<meta charset="utf-8">
<meta http-equiv="x-ua-compatible" content="ie=edge">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title> | PolitiFact </title>
<meta name="description" content="PolitiFact is a fact-checking website that rates the accuracy of claims by elected officials and others on its Truth-O-Meter." />
<meta name="twitter:username" content="PolitiFact" />
<meta name="twitter:card" content="summary" />
<meta


In [9]:
# 1st page BeautifulSoup object, confirm type
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

# html overview

In [37]:
statement_containers = html_soup.find_all('article', class_="o-listicle__content")
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
1


In [70]:
# check html overview
statement_containers

[<li class="o-listicle__item">
 <article class="m-statement m-statement--is-medium m-statement--fire">
 <div class="m-statement__author">
 <div class="m-statement__avatar">
 <div class="m-statement__image">
 <div class="c-image" style="padding-top: 119.27710843373494%;">
 <img class="c-image__thumb" height="99" src="https://static.politifact.com/CACHE/images/politifact/mugs/Facebook_Logo_2019/3b3162a610bc8b3cbeaa2f750c7b6868.jpg" width="83"/>
 <picture>
 <img class="c-image__original" height="178" src="https://static.politifact.com/CACHE/images/politifact/mugs/Facebook_Logo_2019/7cf81e6f294db8af7f47a3cb47f42fc2.jpg" width="166"/>
 </picture>
 </div>
 </div>
 </div>
 <div class="m-statement__meta">
 <a class="m-statement__name" href="/personalities/facebook-posts/" title="Facebook posts">
 Facebook posts
 </a>
 <div class="m-statement__desc">
 stated on May 19, 2021 in a Facebook post:
 </div>
 </div>
 </div>
 <div class="m-statement__content">
 <div class="m-statement__body">
 <div cla

# Hone in on target containers

In [116]:
# finding each row of data we want to scrape, confirm type, check length
def get_statement_containers(html_soup):
    return html_soup.find_all('li', class_='o-listicle__item')

statement_containers = get_statement_containers(html_soup)
print(type(statement_containers))
print(len(statement_containers))

<class 'bs4.element.ResultSet'>
30


In [83]:
# check data
one_container = statement_containers[3]
one_container

<li class="o-listicle__item">
<article class="m-statement m-statement--is-medium m-statement--false">
<div class="m-statement__author">
<div class="m-statement__avatar">
<div class="m-statement__image">
<div class="c-image" style="padding-top: 119.27710843373494%;">
<img class="c-image__thumb" height="99" src="https://static.politifact.com/CACHE/images/politifact/mugs/Facebook_Logo_2019/3b3162a610bc8b3cbeaa2f750c7b6868.jpg" width="83"/>
<picture>
<img class="c-image__original" height="178" src="https://static.politifact.com/CACHE/images/politifact/mugs/Facebook_Logo_2019/7cf81e6f294db8af7f47a3cb47f42fc2.jpg" width="166"/>
</picture>
</div>
</div>
</div>
<div class="m-statement__meta">
<a class="m-statement__name" href="/personalities/facebook-posts/" title="Facebook posts">
Facebook posts
</a>
<div class="m-statement__desc">
stated on May 10, 2021 in a Facebook post:
</div>
</div>
</div>
<div class="m-statement__content">
<div class="m-statement__body">
<div class="m-statement__quote-w

In [84]:
# statement
def get_statement(one_container):
    return one_container.find('div', class_='m-statement__quote').get_text(strip=True)

get_statement(one_container)

'"When you can\'t find gas, remember who executively ordered the Keystone Pipeline to close."'

In [85]:
# statement source
def get_source(one_container):
    return one_container.find('div', class_='m-statement__meta').a["title"]
    
get_source(one_container)

'Facebook posts'

In [107]:
# statement year
def get_year(one_container):
    return one_container.find('div', class_='m-statement__desc').get_text(strip=True)
    
get_year(one_container)

'stated on May 10, 2021 in a Facebook post:'

In [90]:
# statement veracity
def get_veracity(one_container):
    return one_container.find('div', class_='m-statement__meter').img["alt"]

get_veracity(one_container)

'false'

# Test 1 page of data scraping

In [109]:
df = pd.DataFrame(columns=["statement", "source", "year", "veracity"])

for idx, c in enumerate(statement_containers):
    df.loc[idx] = [get_statement(c), get_source(c), get_year(c), get_veracity(c)]

df

,statement,source,year,veracity
0,Photos show Joe Biden didn’t drive himself dur...,Facebook posts,"stated on May 19, 2021 in a Facebook post:",pants-fire
1,"The Biden White House has ""no one there. They ...",Eric Trump,"stated on May 18, 2021 in a Fox News appearance:",pants-fire
2,“Video shows U.S. Capitol Police gave proteste...,Facebook posts,"stated on May 16, 2021 in a Facebook post:",pants-fire
3,"""When you can't find gas, remember who executi...",Facebook posts,"stated on May 10, 2021 in a Facebook post:",false
4,On supporting Liz Cheney for House Republican ...,Mike Gallagher,"stated on May 11, 2021 in Public statement:",full-flop
5,“Watching the TV footage of those who entered ...,Andrew Clyde,"stated on May 12, 2021 in a House hearing:",pants-fire
6,After the American Rescue Plan’s food and nutr...,Joe Biden,"stated on May 5, 2021 in remarks:",mostly-true
7,“First time in 70 years the National Day Of Pr...,Facebook posts,"stated on May 6, 2021 in a Facebook post:",false
8,With the goal to preserve 30% of the earth’s l...,Survival International,"stated on April 22, 2021 in a website press re...",barely-true
9,"""There’s no strings attached to that money tha...",Jon Erpenbach,"stated on March 21, 2021 in TV interview:",half-true


In [110]:
len(df)

30

# Test 2 pages (and check actual website to confirm data)

In [117]:
# If problem-free, proceed to scrape all pages
# Scraping 834 pages takes about 4-5 hours --> 2019 times

# Preparing the monitoring of the loop
start_time = time()
requests = 0
df = pd.DataFrame(columns=["statement", "source", "year", "veracity"])

# For every page in the interval
pages = [i for i in range(1, 2+1)]

for page in pages:
    # Make a get request
    response = get(f'https://www.politifact.com/factchecks/list/?page={page}&category=truth-o-meter')

    # Pause the loop in random intervals so your IP address doesn't get banned
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Parse the content of the request with BeautifulSoup
    html_soup = BeautifulSoup(response.text, 'html.parser')

    # Select all the containers from a single page
    statement_containers = get_statement_containers(html_soup)
    
    for idx, c in enumerate(statement_containers):
        df.loc[f"{page}_{idx}"] = [get_statement(c), get_source(c), get_year(c), get_veracity(c)]
        
time() - start_time

Request:2; Frequency: 0.09252507500931167 requests/s


In [118]:
df

,statement,source,year,veracity
1_0,Photos show Joe Biden didn’t drive himself dur...,Facebook posts,"stated on May 19, 2021 in a Facebook post:",pants-fire
1_1,"The Biden White House has ""no one there. They ...",Eric Trump,"stated on May 18, 2021 in a Fox News appearance:",pants-fire
1_2,“Video shows U.S. Capitol Police gave proteste...,Facebook posts,"stated on May 16, 2021 in a Facebook post:",pants-fire
1_3,"""When you can't find gas, remember who executi...",Facebook posts,"stated on May 10, 2021 in a Facebook post:",false
1_4,On supporting Liz Cheney for House Republican ...,Mike Gallagher,"stated on May 11, 2021 in Public statement:",full-flop
1_5,“Watching the TV footage of those who entered ...,Andrew Clyde,"stated on May 12, 2021 in a House hearing:",pants-fire
1_6,After the American Rescue Plan’s food and nutr...,Joe Biden,"stated on May 5, 2021 in remarks:",mostly-true
1_7,“First time in 70 years the National Day Of Pr...,Facebook posts,"stated on May 6, 2021 in a Facebook post:",false
1_8,With the goal to preserve 30% of the earth’s l...,Survival International,"stated on April 22, 2021 in a website press re...",barely-true
1_9,"""There’s no strings attached to that money tha...",Jon Erpenbach,"stated on March 21, 2021 in TV interview:",half-true


In [119]:
len(df)

60

In [120]:
%%time

# Preparing the monitoring of the loop
start_time = time()
requests = 0
df = pd.DataFrame(columns=["statement", "source", "year", "veracity"])

# For every page in the interval
pages = [i for i in range(1, 185+1)]

for page in pages:
    # Make a get request
    response = get(f'https://www.politifact.com/factchecks/list/?page={page}&category=truth-o-meter')

    # Pause the loop in random intervals so your IP address doesn't get banned
    sleep(randint(8,15))

    # Monitor the requests
    requests += 1
    elapsed_time = time() - start_time
    print('Request:{}; Frequency: {} requests/s'.format(requests, requests/elapsed_time))
    clear_output(wait = True)

    # Throw a warning for non-200 status codes
    if response.status_code != 200:
        warn('Request: {}; Status code: {}'.format(requests, response.status_code))

    # Parse the content of the request with BeautifulSoup
    html_soup = BeautifulSoup(response.text, 'html.parser')

    # Select all the containers from a single page
    statement_containers = get_statement_containers(html_soup)
    
    for idx, c in enumerate(statement_containers):
        df.loc[f"{page}_{idx}"] = [get_statement(c), get_source(c), get_year(c), get_veracity(c)]

2397.15784406662

## only took 40 minutes?

In [121]:
len(df)

5460

In [122]:
# save dataframe to csv
df.to_csv('/Users/victor/Desktop/Product_Analytics/data/politifact_data.csv')

In [134]:
# Tutorial followed to create this scraper: 
# https://www.dataquest.io/blog/web-scraping-beautifulsoup/